# Metagenomic-based Diagnostic for Sepsis (External Validation)

In [1]:
# Import Statements
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

cwd = Path.cwd()
datasets = cwd / 'datasets'
results = cwd / 'results/external_validation'

In [2]:
list(datasets.glob('*'))

[PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/karius_SraRunInfo.csv'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/kapusta_metadata_290620.tsv'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/contaminant_list.txt'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/PRJEB21872.txt'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/PRJEB30958.txt'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/karius_genus_raw_kuniq.csv'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/.RData'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/karius_parsed_metadata.csv'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/PRJEB13247.txt'),
 PosixPath('/ugi/home/zcbtct0/Polymicrobial-Signature-of-Sepsis/datasets/simple_decontam_pathogens.csv'),
 PosixPath('/ugi/home/zcbtct0/Pol

## Data Preprocessing
Since we are using stratified kfold, a validation split is not necesssary.

### Load data

In [3]:
raw_df = pd.read_csv(datasets / 'kapusta_grumaz_karius_genus_raw.csv')

# Remove NTCs
raw_df = raw_df.loc[raw_df.y != 'ntc', :]
display(raw_df)

X = raw_df.drop('y', axis=1).copy()
y = raw_df.y.copy()

,y,Bifidobacterium,Arthrobacter,Kocuria,Cellulomonas,Brevibacterium,Agromyces,Microbacterium,Demequina,Corynebacterium,...,Nosocomiicoccus,Solimonas,Myroides,Bosea,Microvirgula,Myxococcus,Thermoanaerobacter,Collimonas,Eggerthella,Rubrivivax
0,healthy,654.0,0.0,0.0,92.0,19.0,0.0,0.0,2.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,septic,5.0,1.0,0.0,0.0,5.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,healthy,451.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,healthy,763.0,1.0,0.0,2.0,29.0,0.0,34.0,3.0,253.0,...,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0
4,healthy,443.0,0.0,0.0,0.0,24.0,0.0,0.0,5.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,septic,0.0,2.0,1.0,24.0,1.0,0.0,4.0,0.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
195,septic,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
196,septic,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
197,septic,0.0,0.0,39.0,14.0,0.0,2.0,10.0,0.0,3.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
# Binary encode y
y.loc[y == 'septic'] = 1
y.loc[y == 'healthy'] = 0
y = y.astype('int')

# Relative abundance
X_RA = X.apply(func=lambda x: x / x.sum(), axis=1)

In [5]:
n_splits = 10

pos = len(y[y == 1])
neg = len(y[y == 0])
split_sizes = pd.DataFrame({'Septic': [pos - int(pos / n_splits), int(pos / n_splits)], 
                           'Healthy': [neg - int(neg / n_splits), int(neg / n_splits)]}, index=['Train fold', 'Test fold'])

display(split_sizes)

# Get negative to positive ratio
ratio = sum(y == 0) / sum(y == 1)

,Septic,Healthy
Train fold,118,59
Test fold,13,6


## Nested CV for hyperparameter optimisation

In [6]:
# Metrics
from sklearn.metrics import make_scorer, precision_score, recall_score
from sklearn.model_selection import cross_validate

In [7]:
def optimise(X, y, param_dict=False):
    np.random.seed(66)
    
    # Hyperparemeter Optimisation using grid search (F1)
    model = XGBClassifier()
    n_estimators = range(50, 500, 1)
    max_depth = range(1, 10)
    gamma = np.linspace(0, 5, 20)
    subsample = np.linspace(0.1, 1, 20)
    colsample_bytree = np.linspace(0.1, 1, 20)
    
    param_grid = dict(max_depth=max_depth, 
                      n_estimators=n_estimators, 
                      colsample_bytree=colsample_bytree,
                      gamma=gamma,
                      subsample=subsample,
                      scale_pos_weight=[ratio])
    
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True)
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True)
    
    if not param_dict:
        # Inner CV
        model = RandomizedSearchCV(model, 
                                   param_grid, 
                                   scoring="roc_auc",
                                   n_iter=1000,
                                   n_jobs=60, 
                                   cv=inner_cv, 
                                   verbose=1)

        model.fit(X, y)
        best_params = model.best_params_
        print(best_params)
                
    else:
        model = XGBClassifier(**param_dict)
        model.fit(X, y)
        best_params = param_dict

    # Custom metrics
    precision = make_scorer(precision_score, average='binary')
    recall = make_scorer(recall_score, average='binary')
    scoring = {'precision': precision, 
               'recall': recall, 
               'AUROC': 'roc_auc'}
    
    # Outer CV
    outer_results = cross_validate(model, X=X, y=y, cv=outer_cv, scoring=scoring)
    outer_results = pd.DataFrame(outer_results).mean()[['test_precision', 'test_recall', 'test_AUROC']]
    
    return model, outer_results, best_params

### Optimise Model using Neat Data

In [ ]:
raw_model, raw_results, raw_params = optimise(X, y)
# raw_params = {'subsample': 0.5263157894736842, 'scale_pos_weight': 1.4273504273504274, 'n_estimators': 96, 'max_depth': 2, 'gamma': 1.8421052631578947, 'colsample_bytree': 0.19473684210526315}
# raw_model, raw_results, _ = optimise(X, y, raw_params)
RA_model, RA_results, RA_params = optimise(X_RA, y)
# RA_params = {'subsample': 0.4789473684210527, 'scale_pos_weight': 1.4273504273504274, 'n_estimators': 255, 'max_depth': 1, 'gamma': 0.7894736842105263, 'colsample_bytree': 0.33684210526315794}
# RA_model, RA_results, _ = optimise(X_RA, y, RA_params)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    2.3s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:    3.7s
[Parallel(n_jobs=60)]: Done 680 tasks      | elapsed:    5.8s
[Parallel(n_jobs=60)]: Done 1130 tasks      | elapsed:    8.5s
[Parallel(n_jobs=60)]: Done 1680 tasks      | elapsed:   12.0s
[Parallel(n_jobs=60)]: Done 2330 tasks      | elapsed:   16.0s
[Parallel(n_jobs=60)]: Done 3080 tasks      | elapsed:   20.5s
[Parallel(n_jobs=60)]: Done 3930 tasks      | elapsed:   25.9s
[Parallel(n_jobs=60)]: Done 4880 tasks      | elapsed:   31.5s
[Parallel(n_jobs=60)]: Done 5000 out of 5000 | elapsed:   32.5s finished


{'subsample': 1.0, 'scale_pos_weight': 0.4961832061068702, 'n_estimators': 390, 'max_depth': 7, 'gamma': 0.2631578947368421, 'colsample_bytree': 0.1}
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    0.9s
[Parallel(n_jobs=60)]: Done 540 tasks      | elapsed:    2.8s
[Parallel(n_jobs=60)]: Done 1240 tasks      | elapsed:    5.6s
[Parallel(n_jobs=60)]: Done 2140 tasks      | elapsed:    9.1s
[Parallel(n_jobs=60)]: Done 3240 tasks      | elapsed:   13.4s
[Parallel(n_jobs=60)]: Done 4540 tasks      | elapsed:   18.2s
[Parallel(n_jobs=60)]: Done 5000 out of 5000 | elapsed:   20.1s finished


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    0.8s
[Parallel(n_jobs=60)]: Done 540 tasks      | elapsed:    2.7s
[Parallel(n_jobs=60)]: Done 1240 tasks      | elapsed:    5.5s
[Parallel(n_jobs=60)]: Done 2140 tasks      | elapsed:    9.3s
[Parallel(n_jobs=60)]: Done 3240 tasks      | elapsed:   13.6s
[Parallel(n_jobs=60)]: Done 4540 tasks      | elapsed:   18.7s
[Parallel(n_jobs=60)]: Done 5000 out of 5000 | elapsed:   20.5s finished


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.


## Estimates of test error

In [ ]:
metric_df = pd.DataFrame({'Raw': raw_results, 'RA': RA_results}).round(3).T
display(metric_df)

### Remove Contaminants based on SHAP values

In [ ]:
import math
from scipy.stats import spearmanr
import shap


def decontam(X_train, y_train, params):
    model = XGBClassifier(**params)
    model.fit(X=X_train, y=y_train)

    explainer = shap.TreeExplainer(model, feature_pertubation='interventional', model_output='probability', data=X_train)
    shap_val = explainer.shap_values(X_train)

    to_retain = np.array([True] * X_train.shape[1])
    corrs = np.zeros(X_train.shape[1])

    for i in range(X_train.shape[1]):
        rho = spearmanr(X_train.iloc[:, i], shap_val[:, i])[0]
        p = spearmanr(X_train.iloc[:, i], shap_val[:, i])[1]
        if rho < 0 and p < 0.05:
            to_retain[i] = False

        if math.isnan(rho):
            corrs[i] = 2
        else:
            corrs[i] = rho

    to_retain = np.logical_and(corrs > 0, corrs != 2)
    to_retain = X_train.columns[to_retain]
    print(to_retain.shape, to_retain)
    
    return to_retain

In [ ]:
# Decontam using raw_params
genera_new = X.columns

for _ in range(10):
    genera_new = decontam(X.loc[:, genera_new], y, raw_params)

### Remove non-human associated pathogens

In [ ]:
# Retrieve known human pathogens
meta = pd.read_csv(datasets / 'pathogen_list.csv', encoding= 'unicode_escape')
meta = meta['Genus'].unique()

to_retain = list(set(genera_new).intersection(set(meta)))
print(to_retain)

In [ ]:
# Decontam + pathogens
raw_CR = X[to_retain]


In [ ]:
# Get SHAP summary before removing Cellulomonas and Agrobacterium
pre_model = XGBClassifier(**raw_params)
pre_model.fit(X=raw_CR, y=y)

pre_explainer = shap.TreeExplainer(pre_model, feature_pertubation='interventional', model_output='probability', data=raw_CR)
shap_pre = pre_explainer.shap_values(raw_CR)

shap.summary_plot(shap_pre, raw_CR, show=False, plot_size=(4, 5), color_bar_label='Unique k-mer Count', max_display=25)
fig, ax = plt.gcf(), plt.gca()
ax.set_xlabel('SHAP Value')
plt.savefig(results / 'pre_shap.png', dpi=600, format='png', bbox_inches='tight')


### Drop features

In [ ]:
# Normalise Datasets
RA_CR = raw_CR.apply(func=lambda x: x / x.sum(), axis=1)

### Number of Features

In [ ]:
print('Neat', X.shape)
print('CR', raw_CR.shape)

### Optimise decontaminated models

#### Pathogens

In [ ]:
# raw_CR_model, raw_CR_results, raw_CR_params = optimise(raw_CR, y)
raw_CR_params = {'subsample': 0.7631578947368421, 'scale_pos_weight': 1.4273504273504274, 'n_estimators': 426, 'max_depth': 1, 'gamma': 0.0, 'colsample_bytree': 0.1}
raw_CR_model, raw_CR_results, _ = optimise(raw_CR, y, raw_CR_params)
# RA_CR_model, RA_CR_results, RA_CR_params = optimise(RA_CR, y)
RA_CR_params = {'subsample': 0.38421052631578945, 'scale_pos_weight': 1.4273504273504274, 'n_estimators': 101, 'max_depth': 5, 'gamma': 2.894736842105263, 'colsample_bytree': 0.19473684210526315}
RA_CR_model, RA_CR_results, _ = optimise(RA_CR, y, RA_CR_params)

In [ ]:
metric_CR_df = pd.DataFrame({'Raw CR': raw_CR_results, 'RA CR': RA_CR_results}).round(3).T
metric_df = metric_df.append(metric_CR_df)
display(metric_df)

In [ ]:
300620_STOPPPPP

## Interpreting model using SHAP values

### Plot of SHAP values per Feature

In [ ]:
import matplotlib.pyplot as plt
explainer_CR = shap.TreeExplainer(raw_CR_model, feature_pertubation='interventional', model_output='probability', data=raw_CR)
shap_CR = explainer_CR.shap_values(raw_CR)

explainer_raw = shap.TreeExplainer(raw_model, feature_pertubation='interventional', model_output='probability', data=X)
shap_raw = explainer_raw.shap_values(X)

In [ ]:
shap.summary_plot(shap_CR, raw_CR, show=False, plot_size=(4, 5), color_bar_label='Unique k-mer Count', max_display=35)
fig, ax = plt.gcf(), plt.gca()
ax.set_xlabel('SHAP Value')
plt.savefig(results / 'CR_shap.png', dpi=600, format='png', bbox_inches='tight')

In [ ]:
shap.summary_plot(shap_raw, X, show=False, plot_size=(4, 5), color_bar_label='Unique k-mer Count', max_display=23)
fig, ax = plt.gcf(), plt.gca()
ax.set_xlabel('SHAP Value')
plt.savefig(results / 'raw_shap.png', dpi=600, format='png', bbox_inches='tight')

* Features are ranked by importance from top to botttom
* feature values are the kmer counts for each genus
* SHAP values are the average marginal contributions to probability

### Force plot for healthy patient

In [ ]:
j = 201
print(f'Actual Classification {y[j]}')
print(raw_CR.index[j])
shap.force_plot(explainer_CR.expected_value, 
                shap_CR[j,:], 
                raw_CR.iloc[j,:],
                show=False,
                matplotlib=True)
plt.savefig(results / 'CR_force_plot.png', dpi=600, format='png', bbox_inches='tight')

## How much does Escherichia drive predictions?

In [ ]:
raw_CR.columns

In [ ]:
no_ecoli_model, no_ecoli_results, no_ecoli_params = optimise(raw_CR.drop('Escherichia', axis=1), y)

In [ ]:
no_ecoli_results